In [16]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

from bravado.client import SwaggerClient
from bravado.exception import HTTPError

%load_ext autoreload

%autoreload 2
import helper
from Station import Station

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Import client modules
These let us interface with the Swagger Client and pull info from EVE's API

In [2]:
# Lookup type and region info form the SDE
sde_client = SwaggerClient.from_url("http://evekit-sde.orbital.enterprises/latest/swagger.json",
                                   config={'ues_models': False, 'validate_responses' : False})

# We need a SwaggerClient for the market data colleciton service
mdc_client = SwaggerClient.from_url("https://evekit-market.orbital.enterprises//swagger",
                                   config={'use_models': False, 'validate_responses' : False})

### Specify stations and items of interest

In [6]:
# Choose items and stations of interest
# Station Names
names_station = ['Jita IV - Moon 4 - Caldari navy Assembly Plant', 
                 'Amarr VIII (Oris) - Emperor Family Academy']
names_station_shorthand = ['Jita 4-4', 'Amarr']
dict_names = dict(zip(names_station, names_station_shorthand))
# Region names
names_region = ['The Forge', 'Domain']
# Item names
names_items = ["Biocells","Construction Blocks","Consumer Electronics"]

# Date range of interest (generally just today's date for simple trade find)
date = datetime.date.today() - datetime.timedelta(days = 1)

In [18]:
stations = [Station(sde_client, key, value) for key, value in dict_names.items()]

In [22]:
for station in stations:
    print(f"Station Name     : %s"%station.get_station_name())
    print(f"Station Shorthand: %s"%station.get_station_shorthand())
    print(f"Station ID       : %d"%station.get_station_id())
    print(f"Station Region ID: %s"%station.get_region_id())
    print()

Station Name     : Jita IV - Moon 4 - Caldari navy Assembly Plant
Station Shorthand: Jita 4-4
Station ID       : 60003760
Station Region ID: 10000002

Station Name     : Amarr VIII (Oris) - Emperor Family Academy
Station Shorthand: Amarr
Station ID       : 60008494
Station Region ID: 10000043



In [ ]:
# Look up the relative IDs for the names above
dict_station = {x: helper.get_station_id(sde_client, x) for x in names_station}
dict_region  = {x: helper.get_region_id(sde_client, x) for x in names_region}
dict_items   = {x: helper.get_type_id(sde_client, x) for x in names_items}

In [ ]:
# Constuct the master dataframe to house everything
df_master = pd.DataFrame(list(dict_items.keys()))
df_master.columns = ['Items']
df_master.head()

In [ ]:
# Add column for prices of each item for all stations listed, as well as internal spreads
for i in range(0, len(names_station)):
    # Get region for the station
    station_id = helper.get_station_id(sde_client, names_station[i])
    region_id = helper.get_stations_region_id(sde_client, names_station[i])
    test_series = df_master.apply(lambda row:
                        helper.get_item_book(mdc_client,
                                            dict_items[row['Items']],
                                            station_id,
                                            region_id,
                                            date), axis=1)
#     df_master[names_station_shorthand[i]] = df_master.apply(lambda row:
#                         helper.get_item_book(mdc_client,
#                                             dict_items[row['Items']],
#                                             station_id,
#                                             region_id,
#                                             date), axis=1)
    spread_name = f"%s: internal spread"%names_station_shorthand[i]
    buy_name = f"%s: buys"%names_station_shorthand[i]
    sell_name = f"%s: sells"%names_station_shorthand[i]
    df_master[[spread_name,buy_name,sell_name]] = pd.DataFrame(test_series.values.tolist(), 
                        index=df_master.index)

In [ ]:
df_master.head()

In [ ]:
df_ids['jita_spread'] = df_ids.apply(lambda row: 
                helper.get_item_book(mdc_client, 
                                     row['item_IDs'],
                                     row['Jita 4-4'],
                                     row['The Forge'], date), axis=1)
# df_ids['jita_spread'] = df_ids.apply(lambda row: test_print(row['item_IDs'], row['Jita 4-4']), axis=1)

In [ ]:
df_ids.head()

In [ ]:
df_ids['jita_spread'].values

In [ ]:
# Convert IDs to values
helper.get_item_history(mdc_client, )